In [38]:
import pandas as pd
import numpy as np
import math
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import mean_absolute_percentage_error

In [2]:
import utils.data_preprocessing as dp

# データの読み込み

In [3]:
train = pd.read_csv("../data/train.csv")
test = pd.read_csv("../data/test.csv")

In [4]:
train

,id,region,year,manufacturer,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,state,price
0,0,nashville,1949,bmw,excellent,6 cylinders,gas,115148,clean,manual,rwd,mid-size,convertible,orange,NaN,27587
1,1,state college,2013,toyota,fair,8 cylinders,gas,172038,clean,automatic,rwd,full-size,sedan,silver,pa,4724
2,2,wichita,1998,ford,good,6 cylinders,gas,152492,clean,automatic,fwd,full-size,SUV,silver,ks,10931
3,3,albany,2014,ford,excellent,4 cylinders,gas,104118,clean,manual,fwd,mid-size,SUV,blue,ny,16553
4,4,redding,2005,ford,excellent,6 cylinders,gas,144554,clean,manual,fwd,mid-size,sedan,red,ca,5158
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27527,27527,williamsport,2008,ford,good,6 cylinders,gas,26660,clean,automatic,rwd,compact,truck,black,pa,32212
27528,27528,tulsa,2007,ford,excellent,8 cylinders,gas,108072,clean,automatic,rwd,full-size,pickup,black,NaN,5400
27529,27529,rochester,2019,jeep,like new,6 cylinders,gas,139908,clean,automatic,4wd,mid-size,SUV,white,ny,22227
27530,27530,rochester,2007,jeep,excellent,6 cylinders,gas,112326,clean,automatic,4wd,mid-size,sedan,white,ny,3054


In [5]:
train["manufacturer"].value_counts()

ford          6166
chevrolet     3339
bmw           2736
toyota        1570
honda         1526
              ... 
ｓａｔｕｒｎ           1
ＭＩＴＳＵＢＩＳＨＩ       1
lexuѕ            1
ＭＥＲＣＵＲＹ          1
ᴄhrysler         1
Name: manufacturer, Length: 125, dtype: int64

# 定数

In [6]:
area_mapping = {
    # trainデータ
    'SF bay area': 'California',
    'ashtabula': 'Ohio',
    'brainerd': 'Minnesota',
    'brownsville': 'Texas',
    'columbia': 'South Carolina',
    'columbia / jeff city': 'Missouri',
    'daytona beach': 'Florida',
    'dubuque': 'Iowa',
    'el paso': 'Texas',
    'flagstaff / sedona': 'Arizona',
    'florence': 'South Carolina',
    'florida keys': 'Florida',
    'galveston': 'Texas',
    'grand forks': 'North Dakota',
    'grand rapids': 'Michigan',
    'great falls': 'Montana',
    'heartland florida': 'Florida',
    'imperial county': 'California',
    'joplin': 'Missouri',
    'kalispell': 'Montana',
    'lakeland': 'Florida',
    'las vegas': 'Nevada',
    'lawton': 'Oklahoma',
    'long island': 'New York',
    'manhattan': 'New York',
    'merced': 'California',
    'minneapolis / st paul': 'Minnesota',
    'morgantown': 'West Virginia',
    'moses lake': 'Washington',
    'nashville': 'Tennessee',
    'northeast SD': 'South Dakota',
    'northwest KS': 'Kansas',
    'panama city': 'Florida',
    'pittsburgh': 'Pennsylvania',
    'poconos': 'Pennsylvania',
    'pullman / moscow': 'Washington',
    'raleigh / durham / CH': 'North Carolina',
    'rockford': 'Illinois',
    'salem': 'Oregon',
    'san antonio': 'Texas',
    'san diego': 'California',
    'savannah / hinesville': 'Georgia',
    'southern WV': 'West Virginia',
    'southwest VA': 'Virginia',
    'spokane / coeur d\'alene': 'Washington',
    'tallahassee': 'Florida',
    'tucson': 'Arizona',
    'utica-rome-oneida': 'New York',
    'valdosta': 'Georgia',
    'vermont': 'Vermont',
    'waterloo / cedar falls': 'Iowa',
    'watertown': 'New York',
    'western KY': 'Kentucky',
    'yuba-sutter': 'California',
    'yuma': 'Arizona',
    #  testデータ
    'birmingham': 'Alabama',
    'central michigan': 'Michigan',
    'charleston': 'South Carolina',
    'cleveland': 'Ohio',
    'east oregon': 'Oregon',
    'eastern NC': 'North Carolina',
    'glens falls': 'New York',
    'hanford-corcoran': 'California',
    'huntsville / decatur': 'Alabama',
    'jersey shore': 'New Jersey',
    'lafayette': 'Louisiana',
    'muskegon': 'Michigan',
    'ocala': 'Florida',
    'prescott': 'Arizona',
    'rochester': 'New York',
    'siskiyou county': 'California',
    'texarkana': 'Texas',
    'waco': 'Texas',
    'western slope': 'Colorado',
    'wyoming': 'Wyoming'
} 

# 前処理

In [7]:
# manufacturer
train = dp.preprocess_manufacturer(train)

In [8]:
# year
train = dp.fix_year_column(train)

In [9]:
#odometer（負を正に転換）
train["odometer"] = (train["odometer"]**2)**0.5

In [10]:
# size
train = dp.normalize_size_column(train)

In [11]:
# state
dp.fill_missing_state(train, area_mapping)

In [12]:
# fuel, title_status, type（現状、最頻値で埋めている）
dp.fillna_with_mode(train, ['fuel', 'title_status', 'type'])

In [13]:
train.isnull().sum()

id              0
region          0
year            0
manufacturer    0
condition       0
cylinders       0
fuel            0
odometer        0
title_status    0
transmission    0
drive           0
size            0
type            0
paint_color     0
state           0
price           0
dtype: int64

In [14]:
train["odometer"].describe()

count    2.753200e+04
mean     1.160295e+05
std      6.449916e+04
min      1.000000e+00
25%      7.837100e+04
50%      1.114530e+05
75%      1.508562e+05
max      2.946000e+06
Name: odometer, dtype: float64

In [15]:
len(train["state"].value_counts(dropna=False))

79

In [16]:
len(train["region"].value_counts(dropna=False))

372

# 特徴量の追加

In [17]:
# odometerを5000で丸める
# train["odometer_bin5000"]=train["odometer"]//5000
# train.head()

In [18]:
# 人気度をカウント数で算出する（現状これだけだと精度が落ちる）
# df_mts=df_train[["id","manufacturer","type","size"]].groupby(["manufacturer","type","size"]).count().reset_index()
# df_mts.columns=["manufacturer","type","size","mts_counts"]
# df_mts
# train=pd.merge(train,df_mts,on=["manufacturer","type","size"],how="left")

In [19]:
# priceの修正
train["price"]=train["price"].apply(lambda x:math.log10(x))
train

,id,region,year,manufacturer,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,state,price
0,0,nashville,1949,bmw,excellent,6 cylinders,gas,115148.0,clean,manual,rwd,mid-size,convertible,orange,Tennessee,4.440704
1,1,state college,2013,toyota,fair,8 cylinders,gas,172038.0,clean,automatic,rwd,full-size,sedan,silver,pa,3.674310
2,2,wichita,1998,ford,good,6 cylinders,gas,152492.0,clean,automatic,fwd,full-size,SUV,silver,ks,4.038660
3,3,albany,2014,ford,excellent,4 cylinders,gas,104118.0,clean,manual,fwd,mid-size,SUV,blue,ny,4.218877
4,4,redding,2005,ford,excellent,6 cylinders,gas,144554.0,clean,manual,fwd,mid-size,sedan,red,ca,3.712481
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27527,27527,williamsport,2008,ford,good,6 cylinders,gas,26660.0,clean,automatic,rwd,compact,truck,black,pa,4.508018
27528,27528,tulsa,2007,ford,excellent,8 cylinders,gas,108072.0,clean,automatic,rwd,full-size,pickup,black,ok,3.732394
27529,27529,rochester,2019,jeep,like new,6 cylinders,gas,139908.0,clean,automatic,4wd,mid-size,SUV,white,ny,4.346881
27530,27530,rochester,2007,jeep,excellent,6 cylinders,gas,112326.0,clean,automatic,4wd,mid-size,sedan,white,ny,3.484869


In [20]:
# ワンホットに変換
train = pd.get_dummies(train, columns=["condition","manufacturer", "type", "size", "state", "region", "fuel"])
test = pd.get_dummies(test, columns=["condition", "manufacturer", "type", "size", "state", "region", "fuel"])

In [21]:
# trainデータとtestデータのカラムを揃える
missing_cols_train = set(train.columns) - set(test.columns)
for col in missing_cols_train:
    test[col] = 0

missing_cols_test = set(test.columns) - set(train.columns)
for col in missing_cols_test:
    train[col] = 0

# trainデータとtestデータのカラムを並び替える（順序を揃える）
test = test[train.columns]

/tmp/ipykernel_6047/4237110095.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train[col] = 0
/tmp/ipykernel_6047/4237110095.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train[col] = 0
/tmp/ipykernel_6047/4237110095.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train[col] = 0
/tmp/ip

# データの分割

In [22]:
from sklearn.model_selection import train_test_split

In [23]:
# train_train, train_test = train_test_split(train, test_size=0.2,random_state=0)

In [24]:
#priceは予測対象で学習に必要なため別途targetの変数に格納する
target = train["price"]
#学習させる特徴以外を削除
train_set = train.drop(columns=['id', 'cylinders', 'title_status', 'transmission', 'drive', 'paint_color', "price"], axis=1)
test_set = test.drop(columns=['id', 'cylinders',  'title_status', 'transmission', 'drive',  'paint_color', "price"], axis=1)

In [25]:
X_train, X_valid, y_train, y_valid = train_test_split(train_set, target, random_state = 82)
print(X_train.shape, X_valid.shape, y_train.shape, y_valid.shape)

(20649, 611) (6883, 611) (20649,) (6883,)


In [26]:
# len(train_train)

In [27]:
# len(train_test)

In [28]:
# train_test

# 学習と検証

In [29]:
# import pycaret

In [30]:
#ランダムフォレストで学習
model = RandomForestRegressor(random_state=100)
model.fit(X_train, y_train)

RandomForestRegressor(random_state=100)

In [31]:
#予測
pred = model.predict(X_valid)

In [32]:
#評価
score = mean_absolute_percentage_error(y_valid, pred)
print(score*100)

5.458398530831929


# 提出ファイルの出力

In [33]:
predict = model.predict(test_set)
predict

array([4.2375675 , 3.88182012, 3.82445802, ..., 3.93209172, 3.80433785,
       3.80638488])

In [36]:
#submit_sample.csvを読み込み
submit = pd.read_csv("../data/submit_sample.csv", header=None)
submit.head()

,0,1
0,27532,14994.540583
1,27533,10004.210369
2,27534,8000.623545
3,27535,15062.223593
4,27536,8994.715270


In [39]:
submit[1] = np.power(10, predict)
submit.head()

,0,1
0,27532,17280.945444
1,27533,7617.634284
2,27534,6675.103764
3,27535,22556.084259
4,27536,4835.347702


In [40]:
# submitファイルとして保存
submit.to_csv("../submit-file/20230805_rfr_price_log_欠損値埋めとワンホット変換.csv", index=False, header=None)